# Imports

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi, cos, sin, exp
from numpy.linalg import norm
from matplotlib.colors import TwoSlopeNorm
import itertools

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import partial_trace, Statevector, state_fidelity, DensityMatrix, Operator, Pauli
from qiskit_aer.noise import depolarizing_error, pauli_error, NoiseModel
from qiskit_aer.library import save_stabilizer, set_stabilizer, save_density_matrix
from qiskit.visualization import plot_histogram
from qiskit.circuit import Clbit  
from qiskit.circuit.library import Initialize, UnitaryGate




In [17]:
'''Test error-free outcomes for the joint measurements in two-qubit process tomography
   to reveal mistake in Aron's thesis. P^YY_+0 and P^XY_i0 should have +1 as the non-trivial outcome, not -1.'''

# gate definitions
xmatrix = np.array([[0,1], [1,0]], dtype=complex)
zmatrix = np.array([[1,0], [0,-1]], dtype=complex)
imatrix = np.eye(2, dtype=complex)
hmatrix = np.array([[1,1], [1,-1]], dtype=complex) / np.sqrt(2)
ymatrix = np.array([[0, -1j],[1j, 0]], dtype=complex)
smatrix = np.array([[1,0], [0,1j]], dtype=complex)
tmatrix = np.array([[1,0], [0,np.exp(1j*pi/4)]], dtype=complex)
sdagmatrix = np.array([[1,0], [0,-1j]], dtype=complex)

cxmatrix = np.array([
  [1,0,0,0],
  [0,1,0,0],
  [0,0,0,1],
  [0,0,1,0]] , dtype=complex)

# basis states
zero = np.array([1,0], dtype=complex)
one  = np.array([0,1], dtype=complex)
plus = (zero + one)/np.sqrt(2)
i_state = (zero + (1j * one))/np.sqrt(2)

# initial states to test
initial_states = {"pluszero_init": np.kron(plus, zero),
                  "plusi_init":    np.kron(plus, i_state),
                  "izero_init":    np.kron(i_state, zero),
                  "ii_init":       np.kron(i_state, i_state)}


for key in initial_states.keys():
      print(f'\ninitial_state: {key}')

      # apply cx
      psi_init = initial_states[key]
      psi = cxmatrix @ psi_init

      # measure in correct bases
      if key == "pluszero_init":
            YY = np.kron(ymatrix, ymatrix)
            yy_exp = (psi.conj().T @ YY @ psi).real
            print("<YY> =", np.round(yy_exp, 5))  # expect -1.0
            print(f'+1 is erroneous outcome? {np.allclose(yy_exp, -1.0)}')

      elif key == "plusi_init":
            YZ = np.kron(ymatrix, zmatrix)
            yz_exp = (psi.conj().T @ YZ @ psi).real
            print("<YZ> =", np.round(yz_exp, 5))  # expect +1.0
            print(f'-1 is erroneous outcome? {np.allclose(yz_exp, +1.0)}')


      elif key == "izero_init":
            XY = np.kron(xmatrix, ymatrix)
            xy_exp = (psi.conj().T @ XY @ psi).real
            print("<XY> =", np.round(xy_exp, 5))  # expect +1.0
            print(f'-1 is erroneous outcome? {np.allclose(xy_exp, +1.0)}')

      elif key == "ii_init":
            XZ = np.kron(xmatrix, zmatrix)
            xz_exp = (psi.conj().T @ XZ @ psi).real
            print("<XZ> =", np.round(xz_exp))  # expect -1.0
            print(f'+1 is erroneous outcome? {np.allclose(xz_exp, -1.0)}')



initial_state: pluszero_init
<YY> = -1.0
+1 is erroneous outcome? True

initial_state: plusi_init
<YZ> = 1.0
-1 is erroneous outcome? True

initial_state: izero_init
<XY> = 1.0
-1 is erroneous outcome? True

initial_state: ii_init
<XZ> = -1.0
+1 is erroneous outcome? True
